# iDEALParking
## ETL

### Reading data

In [1]:
import pandas as pd  
import numpy as np

data = pd.read_csv('idealistaAPI-2018-Oct-30-1713.csv')
y = data["price"]

### Only in Barcelona

In [3]:
data = data[data['municipality'] == 'Barcelona']
print(data.head())

   Unnamed: 0                        address country detailedType.subTypology  \
0           0  calle del comte d'urgell, 234      es                      NaN   
1           1   calle de tarragona, 149 -157      es                      NaN   
2           2         calle de Rocafort, 116      es                      NaN   
3           3                       mallorca      es                 sedanCar   
4           4           rambla de Volart, 12      es                 sedanCar   

  detailedType.typology  distance        district  exterior  \
0                garage      2106        Eixample     False   
1                garage      2499  Sants-Montjuïc     False   
2                garage      1899        Eixample     False   
3                garage      2007        Eixample     False   
4                garage      3470  Horta Guinardó     False   

     externalReference floor                      ...                       \
0   PLAZA 110 SOTANO B   NaN                      ...   

### Dropping data

In [4]:
data = data.drop("detailedType.typology",1)
data = data.drop("Unnamed: 0",1)
data = data.drop("country",1)
data = data.drop("has360",1)
data = data.drop("has3DTour",1)
data = data.drop("hasVideo",1)
data = data.drop("externalReference",1)
data = data.drop("operation",1)
data = data.drop("propertyCode",1)
data = data.drop("propertyType",1)
data = data.drop("thumbnail",1)
data = data.drop("newDevelopmentFinished",1)
data = data.drop("url",1)
data = data.drop("province",1)
data = data.drop("showAddress",1)
data = data.drop("address",1)
data = data.drop("exterior",1)
data = data.drop("suggestedTexts.title",1)
data = data.drop("suggestedTexts.subtitle",1)
data = data.drop("priceByArea",1)
data = data.drop("latitude",1)
data = data.drop("longitude",1)
data = data.drop("floor",1)
data = data.drop("municipality",1)

### Feature extraction

In [5]:
# Renaming all features
data = data.rename(columns={"detailedType.subTypology":"Type", "neighborhood":"Neighborhood", "distance":"Distance" })
data = data.rename(columns={"numPhotos":"NumPhotos", "newDevelopment":"NewDevelopment", "hasPlan":"HasPlan" })
data = data.rename(columns={"hasLift":"HasLift", "district":"District"})

# Keeping data

# data_k = data
# # District
# data['District'] = (data['District'].astype(str))
# aux = data.copy()
# district = np.unique(data['District'])
# for i in range(0,len(district)):
#     aux[(data['District'] == district[i])] = i+1 
# data['District'] = aux['District']

# # Neighborhood
# data['Neighborhood'] = (data['Neighborhood'].astype(str))
# aux = data.copy()
# neighborhood = np.unique(data['Neighborhood'])
# for i in range(0,len(neighborhood)):
#     aux[(data['Neighborhood'] == neighborhood[i])] = i+1 
# data['Neighborhood'] = aux['Neighborhood']

# # Type
# data = data.dropna()
# data['Type'] = (data['Type'].astype(str))
# aux = data.copy()
# type_ = np.unique(data['Type'])
# for i in range(0,len(type_)):
#     aux[(data['Type'] == type_[i])] = i+1 
# data['Type'] = aux['Type']

# # hasLift
# dummies = [1 if x==True else  0 for x in data["HasLift"]]
# data["HasLift"] = dummies

# # hasPlan
# dummies = [1 if x==True else  0 for x in data["HasPlan"]]
# data["HasPlan"] = dummies

# # newDevelopment
# dummies = [1 if x==True else  0 for x in data["NewDevelopment"]]
# data["NewDevelopment"] = dummies

### Extracting X and y and saving it

In [6]:
y = data["price"]
# data = data.drop("price",1)
# X = np.array(data.values).astype(float)

# # Saving it
# np.save('X',X)
# np.save('y',y)

### Showing data

In [32]:
y[data["District"]=="Eixample"].mean()
district = np.unique(data['District'])
solution = []
for i in range(0,len(district)):
    aux = []
    aux.append(district[i])
    aux.append(y[(data['District'] == district[i])].mean())
    solution.append(aux)
    
# np.save(solution)
solution

[['Ciutat Vella', 104.52],
 ['Eixample', 98.39276485788113],
 ['Gràcia', 94.07035175879398],
 ['Horta Guinardó', 85.87209302325581],
 ['Les Corts', 99.98843930635839],
 ['Nou Barris', 79.8225806451613],
 ['Sant Andreu', 78.51572327044025],
 ['Sant Martí', 151.04109589041096],
 ['Sants-Montjuïc', 87.13304721030043],
 ['Sarrià-Sant Gervasi', 139.0483870967742]]

### First approach: mean along districts

In [33]:
# data

In [36]:
#We add the predicted value to the dataframe 
predicted = []

# data = data.drop("Predicted price by district",1)
district = np.unique(data['District'])
    
for row in data['District']:
    for i in range (0,len(district)):
        if row == district[i]:
            predicted.append(solution[i][1])

data["Predicted price by district"] = predicted



# data


##### We can also add a column with the mean along neighborhoods

In [29]:
neighborhood = np.unique(data['Neighborhood'])
solution2 = []
for i in range(0,len(neighborhood)):
    aux = []
    aux.append(neighborhood[i])
    aux.append(y[(data['Neighborhood'] == neighborhood[i])].mean())
    solution2.append(aux)
    
# np.save(solution)
solution2

[['Baró de Viver', 80.0],
 ['Can Baró', 105.0],
 ['Can Peguera - El Turó de la Peira', 67.5],
 ['Ciutat Meridiana - Torre Baró - Vallbona', 100.0],
 ['Diagonal Mar i el Front Marítim del Poblenou', 78.5],
 ['El Baix Guinardó', 98.66666666666667],
 ['El Besòs', 5074.666666666667],
 ['El Bon Pastor', 62.333333333333336],
 ["El Camp d'En Grassot i Gràcia Nova", 97.8],
 ["El Camp de l'Arpa del Clot", 87.94736842105263],
 ['El Carmel', 85.2],
 ['El Clot', 79.11538461538461],
 ['El Coll', 60.916666666666664],
 ['El Congrés i els Indians', 77.11764705882354],
 ['El Fort Pienc', 95.0],
 ['El Guinardó', 86.05263157894737],
 ['El Gòtic', 108.89473684210526],
 ['El Parc i la Llacuna del Poblenou', 78.89473684210526],
 ['El Poble Sec - Parc de Montjuïc', 75.38095238095238],
 ['El Poblenou', 83.26666666666667],
 ['El Putxet i el Farró', 95.09859154929578],
 ['El Raval', 100.52380952380952],
 ['Horta', 66.0],
 ['Hostafrancs', 97.50847457627118],
 ["L'Antiga Esquerra de l'Eixample", 116.9733333333333

In [37]:
#We add the predicted value to the dataframe 
predicted = []

# data = data.drop("Predicted price by neighborhood",1)
neighborhood = np.unique(data['Neighborhood'])
    
for row in data['Neighborhood']:
    for i in range (0,len(neighborhood)):
        if row == neighborhood[i]:
            predicted.append(solution2[i][1])

data["Predicted price by neighborhood"] = predicted

data

,Type,Distance,District,HasLift,HasPlan,Neighborhood,NewDevelopment,NumPhotos,price,Predicted price by district,Predicted price by neighborhood
0,NaN,2106,Eixample,False,False,L'Antiga Esquerra de l'Eixample,False,8,142.0,98.392765,116.973333
1,NaN,2499,Sants-Montjuïc,False,False,Hostafrancs,False,13,150.0,87.133047,97.508475
2,NaN,1899,Eixample,False,False,La Nova Esquerra de l'Eixample,False,2,38.0,98.392765,90.576000
3,sedanCar,2007,Eixample,True,False,La Nova Esquerra de l'Eixample,False,9,109.0,98.392765,90.576000
4,sedanCar,3470,Horta Guinardó,True,False,El Guinardó,False,1,100.0,85.872093,86.052632
6,carAndMotorcycle,2248,Eixample,False,False,La Sagrada Família,False,4,130.0,98.392765,95.328358
7,sedanCar,2026,Eixample,True,False,L'Antiga Esquerra de l'Eixample,False,3,120.0,98.392765,116.973333
8,compactCar,1414,Eixample,False,False,Sant Antoni,False,4,100.0,98.392765,89.655556
9,sedanCar,2246,Eixample,False,False,Sant Antoni,False,3,80.0,98.392765,89.655556
11,compactCar,1705,Eixample,True,False,La Dreta de l'Eixample,False,3,90.0,98.392765,109.656977
